# Modelo neuronal para predicción de similitud

Dentro de los modelos neuronales, una forma que podemos adoptar es que, al tener ya representaciones vectoriales tanto de los términos como de los documentos, usemos estos vectores como entradas a la red neuronal, de tal forma que ésta prediga la similitud entre el término y el documento.

In [1]:
import torch
import torch.nn as nn
from tqdm import tqdm
from nltk.corpus import brown
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.model_selection import train_test_split
from operator import itemgetter
from collections import Counter

### Preparación del dataset

El data set con el que trabajaremos será supervisado; tendremos:

$$\mattcal{S} = \{(x_{i,j}, y_{i,j}) : x_{i,j} \in \mathbb{R}^d, y_{i,j}\in [0,1]\}$$

donde $x_{i,j}$ será un vector que contenga información del documento $j$ y del término $i$. Este vector lo crearemos al concatenar los vectores de ambos elementos. Los vectores los obtendremos con TFIDF.

Por su parte $y_{i,j}$ es el valor de similitud que conocemos para estos datos. Como en los ejemplos supervisados, aquí debemos tener una supervisión que nos diga que valores esperamos de salida.

In [2]:
#Colección de trabajo
ids = brown.fileids(categories=['romance'])[:5] + brown.fileids(categories=['religion'])[:5]
collection = [' '.join(brown.words(d)) for d in ids]

#Vectorización con TFIDF
vectorizer = TfidfVectorizer(stop_words='english')
X = vectorizer.fit_transform(collection)

#Número de documentos y términos
n_docs, n_terms = X.shape

print(X.shape)

(10, 4077)


Ahora obtenemos los términos y los documentos del modelo de TFIDF.

In [3]:
#Términos
terms = vectorizer.vocabulary_
#Documentos
docs = {d:j for j,d in enumerate(ids)}

Para crear el dataset de entrenamiento, concatenamos los vectores que representan a los documentos y a los términos. En este caso, nuestra supervisión será ingenua, pues por simplicidad tomamos una predicción de 1 si el término está contenido en el documento y 0 si no. Una mejor supervisión podría llevar a mejores resultados.

In [4]:
#Vectores de entrada
x = []
#Supervisión
y = []

for d in ids:
    #Obtiene términos
    words = [word.lower() for word in brown.words(d) if word.lower() in terms.keys()]
    #Obtiene frecuencia de términos
    word_frec = Counter(words)
    for t in terms.keys():
        #Obtiene vectores de término y documento en formato torch
        t_vec, d_vec = torch.Tensor(X.T[terms[t]].todense()), torch.Tensor(X[docs[d]].todense())
        #Concatena los vectores
        vector = torch.cat((d_vec,t_vec), axis=1)
        if word_frec[t] > 1:
            for j in range(word_frec[t]):
                #Obtiene el data set x
                x.append(vector)
                #Obtiene la supervisión si el término 
                #está en el documento
                y.append(1)
        else:
            #Obtiene el data set x
            x.append(vector)
            #Supervisión si el término no está
            #en el documento
            y.append(0)

In [5]:
#Dataset en formato torch
x = torch.stack(x)
y = torch.Tensor(y)
x = x.view(-1,n_terms+n_docs)

print(x,y)

tensor([[0.0000, 0.0000, 0.0000,  ..., 0.0000, 0.0000, 0.0000],
        [0.0000, 0.0000, 0.0000,  ..., 0.0000, 0.0000, 0.0000],
        [0.0000, 0.0000, 0.0000,  ..., 0.0000, 0.0085, 0.0109],
        ...,
        [0.0000, 0.0000, 0.0000,  ..., 0.0000, 0.0000, 0.0224],
        [0.0000, 0.0000, 0.0000,  ..., 0.0000, 0.0000, 0.0224],
        [0.0000, 0.0000, 0.0000,  ..., 0.0000, 0.0000, 0.0224]]) tensor([0., 0., 1.,  ..., 0., 0., 0.])


### Creación y entrenamiento de la red

Nuestra red será una red FeedForward que contendrá cuántas capas ocultas consideremos necesarios. Para definir su arquitectura, usamos la función de Sequential de torch.

La saluda de la red será una sola neurona que utiliza función sigmoide, para que así obtengamos una similitud entre 0 y 1. En este caso, podemos interpretar esta similitud como la probabilidad de que el documento contenga al término.

In [6]:
#Dimensiones de las capas ocultas
dim = 128
#Definimos arquitectura de la red
net = nn.Sequential(nn.Linear(n_docs+n_terms, dim), nn.Tanh(), nn.Linear(dim, 2*dim), nn.Tanh(),
                    nn.Linear(2*dim, 3*dim), nn.ReLU(), nn.Linear(3*dim, 1), nn.Sigmoid())

Para entrenar la red usamos una función de riesgo para clasificación binaria. Definimos también un análisis por minibatches para que sea más rápido el entrenamiento.

In [7]:
%%time

#Función de riesgo
risk = nn.BCELoss()
#Optimizador
optimizer = torch.optim.SGD(net.parameters(), lr=0.1)
#Tamaño de mini batches
batch_size = 100

for epoch in tqdm(range(100)):
    #Permutación de los datos para minibatches
    permutation = torch.randperm(x.size()[0])
    
    for i in range(0,x.size()[0], batch_size):
        #Obtención de los minibatches
        indices = permutation[i:i+batch_size]
        batch_x, batch_y = x[indices], y[indices]
        #Predición
        pred = net(batch_x)
        #Valores reales
        y_real = batch_y.reshape(pred.shape)
        #Cálculo del riesgo
        loss = risk(pred, y_real)
        #Paso backward
        optimizer.zero_grad()
        loss.backward()
        optimizer.step()

100%|██████████| 100/100 [02:27<00:00,  1.48s/it]

CPU times: user 14min 32s, sys: 6.28 s, total: 14min 39s
Wall time: 2min 27s


### Aplicación de la red para predicción de similitud

Finalmente, podemos obtener valores de similitud a partir de la red. Dada un término y un documento, la red nos regresará una probabilidad sigmoide que representará la similitud entre ambos (en base al modelo con el que hemos entrenado).

In [8]:
def forward(term, doc):
    #Genera vector de entrada a partir del término y el documento
    t_vec, doc_vec = X.T[terms[term]], X[docs[doc]]
    x_input = torch.cat( (torch.Tensor(doc_vec.todense()), torch.Tensor(t_vec.todense())), axis=1 )
    
    #Aplica y regresa valores de la red
    return net(x_input)

def consult(term):
    for doc in docs.keys():
        s = forward(term, doc)
        
        yield doc, brown.categories(doc)[0], s.detach().numpy()[0][0]

Podemos ver cómo funciona:

In [9]:
#Resultados de query
result = consult('spirit')

#Imprimir en orden
for r in sorted(result, key=itemgetter(2), reverse=True):
    print(r)

('cd04', 'religion', 0.9367077)
('cp03', 'romance', 0.93440336)
('cd03', 'religion', 0.9342154)
('cp04', 'romance', 0.9336889)
('cd02', 'religion', 0.9312527)
('cd01', 'religion', 0.9304013)
('cp05', 'romance', 0.9300379)
('cp02', 'romance', 0.92487204)
('cp01', 'romance', 0.9155621)
('cd05', 'religion', 0.9044896)
